# DE Zoomcamp 2022 - Homework 5

## In this homework, we will load data into a one node Spark Cluster, run queries on the data and write data out into Parquet Format

In [25]:
import pandas as pd
import pyspark
from pyspark.sql import SparkSession

### Question 1: Run the command "spark.version" What version number is output?

In [26]:
pyspark.__file__, pyspark.__version__

('/home/varunkuntalofficial/spark/spark-3.3.2-bin-hadoop3/python/pyspark/__init__.py',
 '3.3.2')

### Question 2: Repartition the June 2021 HVFHV Data into 12 partitions and save it to Parquet. What is the average size of the Parquet Files?

In [3]:
spark = SparkSession.builder \
    .master("local[*]") \
    .appName('test') \
    .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/03/01 14:34:11 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [78]:
!wget https://github.com/DataTalksClub/nyc-tlc-data/releases/download/fhvhv/fhvhv_tripdata_2021-06.csv.gz

# Converting csv.gz to csv
!gzip -d fhvhv_tripdata_2021-06.csv.gz

# Dropping last column (Not Needed)
# !rev fhvhv_tripdata_2021-06.csv | cut -d "," -f 2- | rev > fhvhv_tripdata_2021-06.csv

In [5]:
!head -n 15 fhvhv_tripdata_2021-06.csv

dispatching_base_num,pickup_datetime,dropoff_datetime,PULocationID,DOLocationID,SR_Flag,Affiliated_base_number
B02764,2021-06-01 00:02:41,2021-06-01 00:07:46,174,18,N,B02764
B02764,2021-06-01 00:16:16,2021-06-01 00:21:14,32,254,N,B02764
B02764,2021-06-01 00:27:01,2021-06-01 00:42:11,240,127,N,B02764
B02764,2021-06-01 00:46:08,2021-06-01 00:53:45,127,235,N,B02764
B02510,2021-06-01 00:45:42,2021-06-01 01:03:33,144,146,N,
B02510,2021-06-01 00:18:15,2021-06-01 00:25:47,49,17,N,
B02510,2021-06-01 00:33:06,2021-06-01 00:42:46,49,225,N,
B02510,2021-06-01 00:46:27,2021-06-01 00:56:50,225,177,N,
B02764,2021-06-01 00:48:06,2021-06-01 01:04:10,209,45,N,B02764
B02875,2021-06-01 00:18:54,2021-06-01 00:26:14,80,256,N,B02875
B02875,2021-06-01 00:31:02,2021-06-01 00:36:39,217,17,N,B02875
B02875,2021-06-01 00:41:53,2021-06-01 01:07:32,17,265,N,B02875
B02875,2021-06-01 00:29:52,2021-06-01 00:54:41,210,76,N,B02875
B02510,2021-06-01 00:15:57,2021-06-01 00:39:36,226,213,N,


In [6]:
!head -n 1001 fhvhv_tripdata_2021-06.csv > head.csv

In [4]:
df_pandas = pd.read_csv("head.csv", index_col=False)

In [8]:
df_pandas.isna().sum()

dispatching_base_num        0
pickup_datetime             0
dropoff_datetime            0
PULocationID                0
DOLocationID                0
SR_Flag                     0
Affiliated_base_number    265
dtype: int64

In [9]:
df_pandas.dtypes

dispatching_base_num      object
pickup_datetime           object
dropoff_datetime          object
PULocationID               int64
DOLocationID               int64
SR_Flag                   object
Affiliated_base_number    object
dtype: object

In [5]:
from pyspark.sql import types

In [56]:
spark.createDataFrame(df_pandas).schema

StructType([StructField('dispatching_base_num', StringType(), True), StructField('pickup_datetime', StringType(), True), StructField('dropoff_datetime', StringType(), True), StructField('PULocationID', LongType(), True), StructField('DOLocationID', LongType(), True), StructField('SR_Flag', StringType(), True)])

In [22]:
df = spark.read.option("header", "true").csv('fhvhv_tripdata_2021-06.csv')

In [23]:
df.head(5)

[Row(dispatching_base_num='B02764', pickup_datetime='2021-06-01 00:02:41', dropoff_datetime='2021-06-01 00:07:46', PULocationID='174', DOLocationID='18', SR_Flag='N', Affiliated_base_number='B02764'),
 Row(dispatching_base_num='B02764', pickup_datetime='2021-06-01 00:16:16', dropoff_datetime='2021-06-01 00:21:14', PULocationID='32', DOLocationID='254', SR_Flag='N', Affiliated_base_number='B02764'),
 Row(dispatching_base_num='B02764', pickup_datetime='2021-06-01 00:27:01', dropoff_datetime='2021-06-01 00:42:11', PULocationID='240', DOLocationID='127', SR_Flag='N', Affiliated_base_number='B02764'),
 Row(dispatching_base_num='B02764', pickup_datetime='2021-06-01 00:46:08', dropoff_datetime='2021-06-01 00:53:45', PULocationID='127', DOLocationID='235', SR_Flag='N', Affiliated_base_number='B02764'),
 Row(dispatching_base_num='B02510', pickup_datetime='2021-06-01 00:45:42', dropoff_datetime='2021-06-01 01:03:33', PULocationID='144', DOLocationID='146', SR_Flag='N', Affiliated_base_number=Non

In [24]:
df.schema

StructType([StructField('dispatching_base_num', StringType(), True), StructField('pickup_datetime', StringType(), True), StructField('dropoff_datetime', StringType(), True), StructField('PULocationID', StringType(), True), StructField('DOLocationID', StringType(), True), StructField('SR_Flag', StringType(), True), StructField('Affiliated_base_number', StringType(), True)])

In [6]:
schema = types.StructType([
    types.StructField('dispatching_base_num', types.StringType(), True),
    types.StructField('pickup_datetime', types.TimestampType(), True),
    types.StructField('dropoff_datetime', types.TimestampType(), True),
    types.StructField('PULocationID', types.IntegerType(), True),
    types.StructField('DOLocationID', types.IntegerType(), True),
    types.StructField('SR_Flag', types.StringType(), True),
    types.StructField('Affiliated_base_number', types.StringType(), True)
])

In [7]:
df = spark.read.option("header", "true").schema(schema).csv('fhvhv_tripdata_2021-06.csv')

In [20]:
df = df.repartition(12)

In [21]:
df.write.parquet('fhvhv/2021/06/')

23/03/01 02:52:01 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers


In [22]:
df.show()

+--------------------+-------------------+-------------------+------------+------------+-------+----------------------+
|dispatching_base_num|    pickup_datetime|   dropoff_datetime|PULocationID|DOLocationID|SR_Flag|Affiliated_base_number|
+--------------------+-------------------+-------------------+------------+------------+-------+----------------------+
|              B02864|2021-06-04 14:18:02|2021-06-04 15:04:26|         231|         138|      N|                B02864|
|              B02884|2021-06-03 12:30:12|2021-06-03 12:41:25|         207|         170|      N|                B02884|
|              B02764|2021-06-03 00:13:00|2021-06-03 00:19:53|          63|          63|      N|                B02764|
|              B02875|2021-06-01 09:31:21|2021-06-01 09:56:41|         242|         169|      N|                B02875|
|              B02872|2021-06-04 21:45:04|2021-06-04 22:08:29|         144|         168|      N|                B02872|
|              B02866|2021-06-03 16:26:2

In [8]:
df.count()

14961892

In [25]:
df.isEmpty()

False

In [9]:
df.printSchema()

root
 |-- dispatching_base_num: string (nullable = true)
 |-- pickup_datetime: timestamp (nullable = true)
 |-- dropoff_datetime: timestamp (nullable = true)
 |-- PULocationID: integer (nullable = true)
 |-- DOLocationID: integer (nullable = true)
 |-- SR_Flag: string (nullable = true)
 |-- Affiliated_base_number: string (nullable = true)



### Question 3: How many taxi trips were started on June 15th?

In [40]:
from pyspark.sql.functions import col, unix_timestamp, desc

In [23]:
filtered_df = df.filter(col("pickup_datetime").startswith("2021-06-15"))

In [24]:
filtered_df.count()

452470

### Question 4: How long is the longest trip in the dataset? 

In [30]:
duration_col = (unix_timestamp(col("dropoff_datetime")) - unix_timestamp(col("pickup_datetime"))).alias("duration")

# Find the maximum duration across all rows in the dataframe
max_duration = df.select(duration_col).agg({"duration": "max"}).collect()[0][0]

# Convert the duration to hours
max_duration_hours = max_duration / 3600

In [31]:
max_duration_hours

66.8788888888889

### Question 5: What port does Spark's User Interface Dashboard run on by default?

In [52]:
spark.sparkContext.uiWebUrl

'http://de-zc.us-central1-c.c.de-zoomcap-project.internal:4040'

### Question 6: What is the name of the most frequent pickup location zone?

In [ ]:
!wget https://github.com/DataTalksClub/nyc-tlc-data/releases/download/misc/taxi_zone_lookup.csv

In [34]:
!head -n 1000 taxi_zone_lookup.csv > "headzone.csv"

In [36]:
hz = pd.read_csv("headzone.csv")
hz.dtypes

LocationID       int64
Borough         object
Zone            object
service_zone    object
dtype: object

In [39]:
hz.head()

,LocationID,Borough,Zone,service_zone
0,1,EWR,Newark Airport,EWR
1,2,Queens,Jamaica Bay,Boro Zone
2,3,Bronx,Allerton/Pelham Gardens,Boro Zone
3,4,Manhattan,Alphabet City,Yellow Zone
4,5,Staten Island,Arden Heights,Boro Zone


In [49]:
zone_lookup_df = spark.read.option("header", "true").csv('taxi_zone_lookup.csv')

In [38]:
zone_lookup_df.printSchema()

root
 |-- LocationID: string (nullable = true)
 |-- Borough: string (nullable = true)
 |-- Zone: string (nullable = true)
 |-- service_zone: string (nullable = true)



In [50]:
joined_df = df.join(zone_lookup_df, df.PULocationID == zone_lookup_df.LocationID)

# Group the rows by the Zone column and count the number of occurrences of each zone
zone_counts = joined_df.groupBy("Zone").count()

# Sort the resulting dataframe by the count in descending order
zone_counts_sorted = zone_counts.sort(desc("count"))

# Select the top row, which corresponds to the most frequent zone
most_frequent_zone = zone_counts_sorted.select("Zone").first()[0]

In [51]:
most_frequent_zone

'Crown Heights North'